In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path 

from tests import *
from pprint import pprint

from plotting import plot_qdc, plot_tdc, plot_testpulse, plot_disc_calibration


/eos/user/j/jowulff/mambaforge/lib/python3.10/site-packages/seaborn/_statistics.py:32: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.4)
  from scipy.stats import gaussian_kde


In [2]:
test_map = {"Aldo": Aldo,
            "TestPulse": TestPulse,
            "ExtTestPulse": ExtTestPulse,
            "DiscCalibration0": DiscCalibration_0,
            "DiscCalibration1": DiscCalibration_1,
            "DiscCalibration2": DiscCalibration_2,
            "DiscCalibration3": DiscCalibration_3,
            "QDCCalibration0": QDCCalibration_0,
            "QDCCalibration1": QDCCalibration_1,
            "QDCCalibration2": QDCCalibration_2,
            "QDCCalibration3": QDCCalibration_3,
            "QDCCalibration4": QDCCalibration_4,
            "QDCCalibration5": QDCCalibration_5,
            "QDCCalibration6": QDCCalibration_6,
            "QDCCalibration7": QDCCalibration_7,
            "TDCCalibration": TDCCalibration}


      
class TestResult:

    def __init__(self,
                path: str | Path = "",
                tests: list[str] = ["Aldo",
                                    "TestPulse",
                                    "ExtTestPulse",
                                    "DiscCalibration0",
                                    "DiscCalibration1",
                                    "DiscCalibration2",
                                    "DiscCalibration3",
                                    "QDCCalibration0",
                                    "QDCCalibration1",
                                    "QDCCalibration2",
                                    "QDCCalibration3",
                                    "QDCCalibration4",
                                    "QDCCalibration5",
                                    "QDCCalibration6",
                                    "QDCCalibration7",
                                    "TDCCalibration"],
                ) -> None:
      
        self.path = path
        self.tests = tests

        # for each test assert that the corresponding file exists
        for test in self.tests:
            datafile = test_map[test](test_result_dir=self.path).datafile
            if not datafile.exists(): 
                raise FileNotFoundError(f"File {datafile} not found in {self.path}")

        self.serial_files = [f for f in Path(self.path).rglob('SN_3*.txt')]
        assert len(self.serial_files) > 0, "No serial files found in the directory"

        self.tester_to_serial = {}
        for f in self.serial_files: 
            sn = int(f.stem.split(" ")[-1])
            with open(f, "r") as file:
                tester = int(file.read().strip())
            self.tester_to_serial[tester] = sn


    def get_data(self,
                test: str,
                filename: str = ""
                ) -> pd.DataFrame:
        if test not in self.tests:
            raise ValueError(f"Test {test} not in {self.tests}")
        
        test_args = {"test_result_dir": self.path,
                    "tester_to_serial": self.tester_to_serial}
        if filename != "":
            test_args["filename"] = filename
            return test_map[test](**test_args).get_data()
        else:
            return test_map[test](**test_args).get_data()

        
    def get_passing_info(self,
                        test: str
                        ) -> pd.DataFrame:
        if test not in self.tests:
            raise ValueError(f"Test {test} not in {self.tests}")

        test_args = {"test_result_dir": self.path,
                    "tester_to_serial": self.tester_to_serial}
        return test_map[test](**test_args).get_passing_info()


In [3]:
test_result_dirs = [d for d in Path("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data").glob("2024*") if d.is_dir()]
# assert that the dirs are sorted by their timestamps (yyyymmddhhmm)
test_result_dirs = sorted(test_result_dirs, key=lambda x: int(x.stem))
test_result_dirs

[PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041556'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041910'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042048'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042204'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407050847'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051014'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051151'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051322'),
 PosixPath

In [4]:
trs = []
for d in test_result_dirs:
    try:
        tr = TestResult(d)
        trs.append(tr)
    except FileNotFoundError as e:
        print(e)
        continue

File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319/aldo.tsv not found
File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320/aldo.tsv not found
File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407081902/fetp_tres_scan.tsv not found
File /eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407171126/aldo.tsv not found


In [5]:
class YieldComputer:
    
    def __init__(self,
                 tests: list,
                 base_dir: str | Path) -> None:
        self.tests = tests
        self.base_dir = base_dir
        

    def get_test_result_dirs(self,
                             base_dir: str):
        test_result_dirs = [d for d in Path(self.base_dir).glob("2024*") if d.is_dir()]
        return sorted(test_result_dirs, key=lambda x: int(x.stem))


    def merge_dataframes_for_test(self,
                                  test: str):
        test_result_dirs = self.get_test_result_dirs(self.base_dir)
        merged_dataframe = pd.DataFrame()
        skipped = []
        for d in test_result_dirs:
            try: 
                tr = TestResult(path=d)
            except FileNotFoundError as e:
                skipped.append(d)
                continue
            data = tr.get_passing_info(test)
            if not merged_dataframe.empty and any(data["SN"].isin(merged_dataframe["SN"])):
                merged_dataframe = merged_dataframe[~merged_dataframe["SN"].isin(data["SN"])]
            merged_dataframe = pd.concat([merged_dataframe, data], ignore_index=True)
        merged_dataframe.set_index("SN", inplace=True)
        #print(f"Skipped {len(skipped)} directories:")
        #pprint(skipped)
        return merged_dataframe


    def get_yield_data(self):
        return pd.concat([self.merge_dataframes_for_test(test).rename(columns={"pass": f"{test}_pass"})
                          for test in self.tests], axis=1)


In [6]:
yc = YieldComputer(tests=["Aldo",
                          "DiscCalibration0",
                          "DiscCalibration1",
                          "DiscCalibration2",
                          "DiscCalibration3",
                          "QDCCalibration0",
                          "QDCCalibration1",
                          "QDCCalibration2",
                          "QDCCalibration3",
                          "QDCCalibration4",
                          "QDCCalibration5",
                          "QDCCalibration6",
                          "QDCCalibration7",
                          "TDCCalibration"],
                   base_dir="/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data")

In [7]:
yield_df = yc.get_yield_data()
for col in yield_df.columns:
    print(f"{col}: {yield_df[col].sum()/len(yield_df)}")


/eos/home-j/jowulff/MTD/combined_plots/tests.py:188: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


Aldo_pass: 0.9622641509433962
DiscCalibration0_pass: 1.0
DiscCalibration1_pass: 1.0
DiscCalibration2_pass: 1.0
DiscCalibration3_pass: 1.0
QDCCalibration0_pass: 1.0
QDCCalibration1_pass: 1.0
QDCCalibration2_pass: 1.0
QDCCalibration3_pass: 1.0
QDCCalibration4_pass: 1.0
QDCCalibration5_pass: 1.0
QDCCalibration6_pass: 0.9905660377358491
QDCCalibration7_pass: 0.9622641509433962
TDCCalibration_pass: 0.9811320754716981


In [8]:
total_yield = (yield_df.sum(axis=1) == len(yield_df.columns)).sum()/len(yield_df)
print(f"Total yield: {total_yield:.2%}")

Total yield: 90.57%


In [9]:
aldo_merged = yc.merge_dataframes_for_test("Aldo")
aldo_merged

/eos/home-j/jowulff/MTD/combined_plots/tests.py:188: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


,pass
SN,
229,True
289,True
290,True
294,True
318,True
...,...
123,True
140,True
177,True


In [12]:
test_results = yc.get_test_result_dirs("/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data")
test_results

[PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041556'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041737'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407041910'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042048'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407042204'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407050847'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051014'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051151'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051319'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051320'),
 PosixPath('/eos/user/a/aboletti/TOFHIR2C_validation/tmp_calibration_data/202407051322'),
 PosixPath

In [14]:
aldo_data = yc.merge_dataframes_for_test("Aldo")
aldo_data

/eos/home-j/jowulff/MTD/combined_plots/tests.py:188: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


,pass
SN,
229,True
289,True
290,True
294,True
318,True
...,...
123,True
140,True
177,True


In [27]:
# set index dtype to int
pd.concat([aldo_data.index.dtype("int")], axis=1).reset_index(drop=True)

,SN,pass
0,135.0,False
1,229.0,True
2,289.0,True
3,290.0,True
4,294.0,True
5,318.0,True
6,334.0,True
7,341.0,True


In [37]:
aldo_data = aldo_data.reset_index(drop=False)
aldo_data

,SN,pass
0,135.0,False
1,229.0,True
2,289.0,True
3,290.0,True
4,294.0,True
5,318.0,True
6,334.0,True
7,341.0,True


In [39]:
aldo_data["SN"] = aldo_data["SN"].astype(int)
aldo_data

,SN,pass
0,135,False
1,229,True
2,289,True
3,290,True
4,294,True
5,318,True
6,334,True
7,341,True


In [31]:
tr1 = TestResult(path=test_results[1])
aldo_data_1 = tr1.get_passing_info("Aldo")
aldo_data_1

SN
106.0    True
135.0    True
219.0    True
291.0    True
295.0    True
319.0    True
333.0    True
Name: pass, dtype: bool

In [21]:
concat_series = pd.concat([aldo_data, aldo_data_1], ignore_index=False)


In [24]:
concat_series.index

Index([135.0, 229.0, 289.0, 290.0, 294.0, 318.0, 334.0, 341.0, 106.0, 135.0,
       219.0, 291.0, 295.0, 319.0, 333.0],
      dtype='float64', name='SN')

In [5]:
merged_dfs = {test_name: pd.concat([tr.get_data(test_name) for tr in trs], ignore_index=True) for test_name in test_map.keys()}

/eos/home-j/jowulff/MTD/combined_plots/tests.py:156: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


In [6]:
merged_dfs.keys()

dict_keys(['Aldo', 'TestPulse', 'ExtTestPulse', 'DiscCalibration0', 'DiscCalibration1', 'DiscCalibration2', 'DiscCalibration3', 'QDCCalibration0', 'QDCCalibration1', 'QDCCalibration2', 'QDCCalibration3', 'QDCCalibration4', 'QDCCalibration5', 'QDCCalibration6', 'QDCCalibration7', 'TDCCalibration'])

In [7]:
merged_dfs["Aldo"]["pass"].sum()/len(merged_dfs["Aldo"])

0.9814814814814815

In [8]:
for test_name, df in merged_dfs.items():
    try:
        print(f"Yield of {test_name}: {df['pass'].sum()/len(df):.2%}")
        print(f"SNs failing: {df[df['pass'] == False]['SN'].unique()}")
    except KeyError as e:
        print(f"skipping {test_name}")
        continue
    

Yield of Aldo: 98.15%
SNs failing: [135. 280. 293. 105. 325.]
skipping TestPulse
skipping ExtTestPulse
Yield of DiscCalibration0: 100.00%
SNs failing: []
Yield of DiscCalibration1: 100.00%
SNs failing: []
Yield of DiscCalibration2: 100.00%
SNs failing: []
Yield of DiscCalibration3: 100.00%
SNs failing: []
Yield of QDCCalibration0: 100.00%
SNs failing: []
Yield of QDCCalibration1: 100.00%
SNs failing: []
Yield of QDCCalibration2: 100.00%
SNs failing: []
Yield of QDCCalibration3: 100.00%
SNs failing: [125]
Yield of QDCCalibration4: 100.00%
SNs failing: []
Yield of QDCCalibration5: 100.00%
SNs failing: []
Yield of QDCCalibration6: 100.00%
SNs failing: [125 108]
Yield of QDCCalibration7: 99.99%
SNs failing: [280 350 327]
Yield of TDCCalibration: 99.99%
SNs failing: [162 109]


In [20]:
aldo_1 = Aldo(test_result_dir=test_result_dirs[0], tester_to_serial=trs[0].tester_to_serial).get_data()
aldo_1

/eos/home-j/jowulff/MTD/combined_plots/tests.py:156: RuntimeWarning: invalid value encountered in double_scalars
  max_inl = max(abs(error)) / slope


,tester_ID,asic_id,side,gain,slope,b,max_inl,SN,pass
0,0,0,0,0,0.020495,35.412772,2.191727,334.0,True
1,0,0,0,1,0.040751,32.309174,4.052352,334.0,True
2,0,0,1,0,0.020592,35.413578,2.137131,334.0,True
3,0,0,1,1,0.041010,32.312781,4.770937,334.0,True
4,0,1,0,0,0.020424,34.736938,1.873895,334.0,True
...,...,...,...,...,...,...,...,...,...
59,7,0,1,1,0.040439,31.727445,3.962144,229.0,True
60,7,1,0,0,0.020484,35.146417,1.407178,229.0,True
61,7,1,0,1,0.040806,32.051225,3.644348,229.0,True
62,7,1,1,0,0.020704,35.190196,1.659040,229.0,True


In [32]:
(sn_s := aldo_1.groupby("SN")["pass"].all())[sn_s == True].index.tolist()

[229.0, 289.0, 290.0, 294.0, 318.0, 334.0, 341.0]

In [9]:
passing_serials_per_test = {}
for test_name, df in merged_dfs.items():
    try:
        passing_serials_per_test[test_name] = df[df["pass"] == True].SN.unique().tolist()
    except KeyError as e:
        print(f"Skipping {test_name}")
        continue

    

Skipping TestPulse
Skipping ExtTestPulse


In [19]:
merged_dfs["Aldo"][merged_dfs["Aldo"]["pass"] == True].SN.unique()

array([ 334.,  290.,  341.,  289.,  294.,  318.,  229.,  135.,  295.,
        291.,  333.,  219.,  319.,  106.,  331.,  337.,  348.,  336.,
        287.,  121.,  332.,  328.,  339.,  320.,  122.,  344.,  147.,
        343.,  317.,  187.,  281.,  342.,  340.,  280.,  350.,  338.,
        162.,  171.,  326.,  349.,  293.,  173.,  282.,  128.,  190.,
        130.,  153.,  322.,  157.,  285.,  192.,  286.,  114.,  193.,
        134.,  195.,  204.,  129.,  152.,  191.,  113.,  345.,  188.,
        105.,  124.,  117.,  160.,  119.,  146.,  133.,  346.,  127.,
        324.,  284.,  137.,  180.,  325.,  176.,  196.,  125.,  109.,
        351.,  116.,  292.,  189.,  185.,  142.,  203.,  194., 9999.,
        212.,  215.,  232.,  321.,  112.,  347.,  327.,  323.,  140.,
        123.,  177.,  101.,  217.,  108.,  103.,  247.])

In [10]:
passing_serials_per_test = {k: set([int(sn) for sn in v]) for k, v in passing_serials_per_test.items()}
all_serials_per_test = {k: set([int(sn) for sn in v.SN.unique()]) for k, v in merged_dfs.items()}

In [11]:
all_serials = set.union(*all_serials_per_test.values())